In [1]:
import numpy as np
import os 
from scipy.linalg import lu_factor, lu_solve

In [2]:
#LU factorization:
def lu_factorization(A, Rhs, x, y):
    n = A.shape[1] + 2
    u_star = np.zeros((n, 1))

    # Perform LU factorization (Doolittle method)
    for i in range(n - 2):
        for j in range(i, n - 2):
            factor = A[j, i] / A[i, i]
            A[j, i:] -= factor * A[i, i:]
            Rhs[j] -= factor * Rhs[i]

    # Solve Ly = Rhs for y using forward substitution
    y = np.zeros(n - 2)
    y[0] = Rhs[0] / A[0, 0]
    for i in range(1, n - 2):
        y[i] = (Rhs[i] - np.dot(A[i, :i], y[:i])) / A[i, i]

    # Solve U*u_star = y for u_star using back substitution
    u_star[-1] = y[-1] / A[-1, -1]
    for i in range(n - 4, -1, -1):
        u_star[i + 1] = (y[i] - np.dot(A[i, i + 1:], u_star[i + 1:])) / A[i, i]

    # Set boundary conditions
    u_star[0] = x
    u_star[-1] = y[-1]

    return u_star



In [3]:
def get_abc(Grid_points,rho,dt,dx,f,A,u,p_s):
    b=np.ones((Grid_points))
    a=np.ones((Grid_points))
    c=np.ones((Grid_points-1))

    for i in range (0,Grid_points):
        a[i]=((rho*A[2*i]/dt)+(f*rho*u[i]*p_s[2*i]/8))
        b[i]=((rho*A[2*i]/(2*dx)))
        if i<Grid_points-1:
            c[i]=(A[(2*i)+1]/dx)             
    return a,b,c        
def get_d(Grid_points,rho,A,u,dt):
    d=np.ones((Grid_points))
    for i in range(0,Grid_points):
        d[i]=((rho*A[2*i]*u[i]/dt))  
    return d             

In [4]:
#for momentum equation final draft
def solve_momentum(a, b, c, d, p_star):
    """
    Calculate u_star vector using the provided inputs and matrix inversion.

    Parameters:
    a (numpy.ndarray): Coefficients a_{i+1/2}^{n+1}.
    b (numpy.ndarray): Coefficients b_{i-1/2}^{n+1} & b_{i+3/2}^{n+1}.
    c (numpy.ndarray): Coefficients c_i^{n+1}.
    d (numpy.ndarray): Coefficients d_{i+1/2}^{n}.
    p_star (numpy.ndarray): Guessed pressure vector.

    Returns:
    u_star (numpy.ndarray): Calculated u_star vector.
    """
    #len of p_star is equal to length of a-1;
    # Calculate the size of the system (number of equations)
    n = len(a)
    x=1
    y=1
    if n>2:
        # Initialize the coefficient matrix A and the right-hand side vector b
        A = np.zeros((n-2, n))
        u_star=np.zeros((n,1))
        Rhs=np.zeros((n-2,1))
        # Populate the coefficient matrix and the right-hand side vector
        for i in range(1,n-1):
            A[i-1, i-1] =b[i-1]
            A[i-1,i] = (-1*a[i])
            A[i-1,i+1]=b[i+1]
            const = ((c[i] * p_star[i]) - (c[i-1] * p_star[i-1]) - d[i])
            Rhs[i-1,0]=const
        u_star=lu_factorization(A, Rhs, x, y)
    return u_star    

In [5]:
#pressure correction coefficients:
def get(Grid_points,u,u_star,A,p_s,rho,dx,dt,f):
    alpha=np.ones(Grid_points-1)
    beta=np.ones(Grid_points-1)
    gama=np.ones(Grid_points-1)
    z=np.ones(Grid_points-1)
    if (Grid_points>3): 
        for i in range(3,(2*Grid_points)-3,2):
            a_prev=(rho*A[i-1]/dx)
            b_prev=(rho*A[i-1]/dt)
            c_prev=(f*rho*u[(i-1)//2]*p_s[i-1]/8) 
            a_after=(rho*A[i+1]/dx)
            b_after=(rho*A[i+1]/dt)
            c_after=(f*rho*u[(i+1)//2]*p_s[i+1]/8)
            gama1=(a_prev/(b_prev+c_prev))
            beta1=(a_after/(b_after+c_after))
            beta[i-3]=beta1*(A[i+2]/dx)
            gama[i-3]=gama1*(A[i-2]/dx)
            alpha[i-3]=(beta1+gama1)*(A[i]/dx)
            z[i-3]=(((rho*A[i-1]/dx)*(u_star[(i-1)//2]))-((rho*A[i+1]/dx)*(u_star[(i+1)//2]))+(((rho*A[i])-(rho*A[i]))/dt))
    return alpha,gama,beta,z


In [6]:
def correct_pressure(Grid_points,alpha,beta,gama,z):
    """
    Calculate P' vector using the provided inputs and matrix inversion.

    Parameters:
    alpha (numpy.ndarray): Coefficients a_{i+1/2}^{n+1}.
    beta (numpy.ndarray): Coefficients b_{i-1/2}^{n+1} & b_{i+3/2}^{n+1}.
    gama (numpy.ndarray): Coefficients c_i^{n+1}.
    z(numpy.ndarray): Coefficients d_{i+1/2}^{n}.

    Returns:
    p' (numpy.ndarray): Calculated p'(It is the value of p that has to be added) vector.
    """
    n=Grid_points
    A = np.zeros((n-2, n))
    p_adjust=np.zeros((n-2,n))
    Rhs1=np.zeros((n-2,1))
    #populate the coefficent matrix
    for i in range(1,n-1):
        A[i-1,i-1] = gama[i-1]
        A[i-1,i] = (-1*alpha[i-1])
        A[i-1,i+1]= (beta[i-1])
        const = z[i-1]
        Rhs1[i-1,0]=const
    return A,Rhs1

    
    

In [7]:
# main algo
def unsteady_1D_flow(A,u,p_star,p_s,Grid_points,rho,dx,dt,f):
    for t in range(0,n):


    #Mometum equation part 
        a_values,b_values,c_values=get_abc(Grid_points,rho,dt,dx,f,A,u,p_s)#solve co-efficients at all grid points
        d_values=get_d(Grid_points,rho,A,u,dt)
        u_star1= solve_momentum(a_values, b_values, c_values, d_values, p_star)# Solve momentum equation to find intermediate velocity (u_star)
    #Pressure Adjusted part
        alpha,beta,gama,z=get(Grid_points,u,u_star,A,p_s,rho,dx,dt,f) 
        p_correct=correct_pressure(Grid_points,alpha,beta,gama,z)#Solve pressure correction equation      
    return u_star1

In [8]:
# Define constants and parameters
n = 1  # Number of time steps
L = 1.0  # Length of the 1D domain
dx = 0.01 # Spatial grid size
dt = 0.01  # Time step size
rho = 1.0 # Density
A = 1.0 # Cross-sectional area
f = 1.0   # Source term
p_s = 1.0  # Viscosity
g = 10.0   # Gravitational acceleration
Grid_points=5 #Total number of grid points 

# Initialize arrays for velocity (u) and pressure (p) at staggered grid locations
u = np.ones(Grid_points)  # Velocity at staggered grid locations
p_star = np.ones(Grid_points-1) # Pressure at cell centers
A=np.ones((2*Grid_points)-1)
p_s=np.ones((2*Grid_points)-1)
u_star= np.ones(Grid_points)


velocity,pressure =unsteady_1D_flow(A,u,p_star,p_s,Grid_points,rho,dx,dt,f)
print(pressure)
print(u_star)







C:\Users\mohan krishna\AppData\Local\Temp\ipykernel_6092\3595939294.py:9: RuntimeWarning: invalid value encountered in scalar divide
  factor = A[j, i] / A[i, i]


IndexError: index 3 is out of bounds for axis 0 with size 3